In [1]:
%matplotlib inline
 
import matplotlib.pyplot as plt
from clawpack import riemann
from clawpack import pyclaw
import numpy as np


def initialize(x,state):
    """
    The initial value.  
    """    
     
    h0  = state.problem_data["initial_depth"]
    l0  = state.problem_data['box_length'] 
     
    dim = state.q.shape[1]   
    for i in range(dim):
        if 0<= x[i]<= l0:
            state.q[0,i] = h0
        else:
            state.q[0,i] = 0.

def wall_bc(state,dim,t,qbc,num_ghost):
    qbc[0, :num_ghost] = -qbc[0, 2 ]  

def Solveur(q_l,q_r,aux_l,aux_r,problem_data):
    r"""
    1d solver with flux f = a u^n with n>=1
    """
    num_eqn = 1
    num_waves = 1
    num_rp = q_l.shape[1]

    # Return values
    wave = np.empty( (num_eqn, num_waves, num_rp) )
    s    = np.empty( (num_waves, num_rp) )
    amdq = np.empty( (num_eqn, num_rp) )
    apdq = np.empty( (num_eqn, num_rp) )

    # Wave
    delta = np.empty(np.shape(q_l))
    delta = q_r - q_l
    wave[0,0,:] = delta
    
    # Parameters
    n   = problem_data['n']
    a   = problem_data['a']
 
    # Compute the shock speed
    if n>1:
        for i in range(num_rp):
            if q_r[0,i] != q_l[0,i]:
                s[0,i] =   a * (q_r[0,i]**n - q_l[0,i]**n)/(q_r[0,i]  - q_l[0,i])
            else:
                s[0,i] = 0
    else:
        s[0,:] = a * np.ones(num_rp)
    
    # Compute the left going and right going fluctuations
    s_index = np.zeros((2,num_rp))
    s_index[0,:] = s[0,:]
    amdq[0,:] = np.min(s_index,axis=0) * wave[0,0,:]
    apdq[0,:] = np.max(s_index,axis=0) * wave[0,0,:]
        
    # Compute entropy fix
    if problem_data['efix']:
        transonic = (q_l[0,:] < 0.0) * (q_r[0,:] > 0.0)
        amdq[0,transonic] = -a * q_l[0,transonic]**n
        apdq[0,transonic] =  a * q_r[0,transonic]**n

    return wave, s, amdq, apdq

def setup(L,mx,tf,nbs,slope,h0,l0,mu,rho):
    
    solver    = pyclaw.ClawSolver1D()
    solver.rp = Solveur
    solver.num_waves = 1
    solver.num_eqn   = 1
    solver.kernel_language = 'Python'
    solver.limiters        = pyclaw.limiters.tvd.superbee
    solver.bc_lower[0]     = pyclaw.BC.custom
    solver.user_bc_lower   = wall_bc
    solver.bc_upper[0]     = pyclaw.BC.extrap
    solver.order           = 2 #1: Godunov, 2: Lax-Wendroff-LeVeque
    
    x = pyclaw.Dimension(0, L, mx, name='x')
    domain = pyclaw.Domain(x)
    num_eqn = 1
    
    state = pyclaw.State(domain, num_eqn)
    
    g = 9.81
    state.problem_data['a']         = rho*np.sin(slope)*g/3/mu
    state.problem_data['density']   = rho
    state.problem_data['n']         = 3
    state.problem_data['efix']      = True
    state.problem_data['viscosity'] = mu
    state.problem_data['initial_depth'] = h0
    state.problem_data['box_length']    = l0
    state.problem_data['slope']         = slope
    
    xc = domain.grid.x.centers
    
    initialize(xc,state)
    #state.q[0,:] = 0.5*np.sin(xc)**2
     
    claw = pyclaw.Controller()
    outdir        = './_output'
    output_style  = 1
    claw.solution = pyclaw.Solution(state, domain)
    claw.solver   = solver
    claw.outdir   = outdir
    claw.output_style = output_style
    
    claw.tfinal           = tf
    claw.num_output_times = nbs
    claw.keep_copy = True
    claw.verbosity = 0
 
    return claw

In [2]:
L = 3 ; mx = 1000 ; tf = 5 ; nbs = 50 ; slope = np.pi/12 ; h0 = 0.5 ; l0 = 1 ; mu = 400 ;  rho = 1000
    
claw = setup(L,mx,tf,nbs,slope,h0,l0,mu,rho)
claw.run()

plt.rcParams['text.usetex'] = True
index = 10
frame = claw.frames[index]
dt = claw.tfinal/claw.num_output_times
t  = dt*index
x  = frame.state.grid.c_centers
x  = x[0]

x_true = np.linspace(0,L,mx)
#q_true = qsol(x_true,t,a)

fig, ax = plt.subplots(figsize=(5, 2.7))
w = frame.q[0,:]
#ax.plot(x_true,q_true, '.',label='g()')
ax.plot(x, w, '--',label='Clawpack sol.')

ax.legend(loc='right')
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$q$')
plt.savefig("HuppertSolution.pdf")

In [3]:
 
def huppert_animation():
    import matplotlib.animation
    import numpy
    # compute the solution with the method define above:
    #claw = setup()
    #claw.keep_copy = True
    #claw.run()
    x = claw.frames[0].grid.dimensions[0].centers
    x_true = numpy.linspace(0., L , mx)
    hn = 0. 
    
    fig = plt.figure()
    axes = plt.subplot(1, 1, 1)
    plt.rcParams['text.usetex'] = True
    axes.set_xlabel(r'$x$',fontsize=14)
    axes.set_ylabel(r'$h(x, t)$',fontsize=14)
    axes.set_xlim((x[0], x[-1]))
    axes.set_ylim((0, 1))
    axes.set_title(" ")
    text = axes.text(L/2, 0.8, '')
    
    def init():

        q_true = np.full_like(x_true,hn)
        computed_line, = axes.plot(x[0], claw.frames[0].q[0, :][0], 'r')
        exact_line, = axes.plot(x_true[0], q_true[0] , 'k--')
        return  (computed_line, exact_line)
    
    computed_line, exact_line = init()
    #computed_line  = init()
    
    def fplot(n):
        tt = claw.frames[n].t
        q_true = np.full_like(x_true,hn)
        computed_line.set_data([x,], [claw.frames[n].q[0, :]])
        exact_line.set_data([x_true], [q_true ])
        
        val = f'{tt:.2f}'
        text.set_text(r'$ t = {} $ s'.format(val))
        #text.set_text(rf't = {tt:.2f} s' )
        return (computed_line, exact_line)

    frames_to_plot = range(0, len(claw.frames))
    plt.close(fig)
    return matplotlib.animation.FuncAnimation(fig, fplot, frames=frames_to_plot, interval=100,
                                   blit=True, init_func=None, repeat=False)

from IPython.display import HTML 
anim = huppert_animation()
HTML(anim.to_jshtml())

In [4]:
# Export mp4
anim.save('KinematicWave_huppert.mp4',fps=5,writer="ffmpeg",dpi=300)
 
    

In [6]:
# Reading files from a previous computation
# solutions files are assumed to be in the _output directory
import os, fnmatch
 
from clawpack.pyclaw.solution import Solution
from clawpack.pyclaw import solution

    
# Finding the home directory
from os.path import expanduser
home = expanduser("~")
cwd = os.getcwd()

user_directory = cwd

dirName = '_output'  #directory in which the results lie
dirAddress = user_directory+"/"+dirName

print("Here are the output files: "+dirAddress)
files=fnmatch.filter(os.listdir(dirAddress), 'fort.q*' )
NbSim = len(files)
print("number of files: "+str(NbSim))

# Alternative to export snapshots
def ExportAnimation():
    figs = []
 
    sol=solution.Solution(1, path=dirAddress, file_format='ascii')
    frame = np.empty((NbSim,)+sol.state.q.shape)
    xsol = np.empty((NbSim, sol.state.q.shape[1]))
    hsol = np.empty((NbSim, sol.state.q.shape[1]))
    tps  = np.empty(NbSim)

    for i in range(0,NbSim):
        sol     = solution.Solution(i, path=dirAddress, file_format='ascii')
        tps[i]  = sol.t
        xsol[i] = sol.state.grid.x.centers
        hsol[i] = sol.state.q[0]

    hmin = 0.0
    hmax = 0.6
    L = max(sol.grid.x.centers)
    x0 = min(sol.grid.x.centers)
    xmoy = (L+x0)/2

 
    for i in range(0,NbSim):
        fig = plt.figure()
        axes = plt.subplot(1, 1, 1)
        plt.rcParams['text.usetex'] = True
        axes.set_xlabel(r'$x$',fontsize=14)
        axes.set_ylabel(r'$h(x, t)$',fontsize=14)
        axes.set_xlim((x0,L))
        axes.set_ylim(( hmin, hmax))
        text = axes.text(L/2, 0.9*hmax , '')
        tt = tps[i]
        val = f'{tt:.2f}'
        text.set_text(r'$ t = {} $ s'.format(val))
        axes.set_title(" ")
        text = axes.text(xmoy, 0.9*hmax , '')
        text = axes.text(xmoy, 0.9*hmax , '')
        axes.plot(xsol[i] , hsol[i] , 'r')
        figs.append(fig)
        plt.close(fig)
    return figs

from clawpack.visclaw import animation_tools
figures = ExportAnimation()
animation_tools.interact_animate_figs(figures)

Here are the output files: /home/ancey/dossier_Clawpack/ADVECTION/Nonlinear_advection_huppert/_output
number of files: 51


interactive(children=(IntSlider(value=0, description='frameno', max=50), Output()), _dom_classes=('widget-inte…

In [7]:
 # export figures for latex
newdir = 'KinematicWaveHuppert'
if not os.path.isdir(newdir): os.mkdir(user_directory+"/"+newdir)
for i in range(len(figures)):
    figures[i].savefig(user_directory+'/'+newdir+'/Frame'+str(i)+'.png', bbox_inches='tight',dpi=300)